In [19]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# Load environment variables from .env file
load_dotenv()

# Fetch API credentials from environment variables
api_key = os.getenv('AZURE_OPENAI_GPT_4O_API_KEY') 
end_point = os.getenv('AZURE_OPENAI_GPT_4O_ENDPOINT')
api_version = os.getenv('AZURE_OPENAI_GPT_4O_API_VERSION')

# Initialize Azure OpenAI client
llm_client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=end_point,
    api_version=api_version
)

# Send a chat completion request to GPT-4o
response = llm_client.chat.completions.create(
    model="gpt-4o",  # Azure model deployment name
    messages=[
        {"role": "system", "content": "You are an AI assistant"},
        {"role": "user", "content": "Tell me about the large language models"},
    ],
    max_tokens=256
    # temperature=0.7  # Uncomment to adjust randomness of responses
)

# Print only the model's reply
print(f"[RESPONSE] {response.choices[0].message.content}")

[RESPONSE] Large Language Models (LLMs) are a type of artificial intelligence (AI) designed to process and generate human-like language. These models are built on deep learning techniques, particularly a subset known as transformers. LLMs have advanced significantly in recent years, achieving remarkable capabilities in natural language understanding and generation. Below, we explore the key aspects of LLMs:

---

### 1. **Core Technology**
   - **Transformers**: LLMs are typically based on the transformer architecture introduced in the landmark 2017 paper "Attention is All You Need" by Vaswani et al. Transformers leverage a mechanism called "self-attention," which evaluates the relationships between different words or tokens in a sequence to understand context and semantics better than older architectures like RNNs or LSTMs.
   - **Pre-training and Fine-tuning**: LLMs are pre-trained on massive datasets of text from the internet (e.g., books, articles, websites) using self-supervised l

In [20]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import chromadb

# Load embedding and cross-encoder models
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="knowledge_base")
collection = chroma_client.get_or_create_collection(name="embedding_collection")

def fetch_top_relevant_queries(query, collection, top_k=10):
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0], results['metadatas'][0], results['distances'][0]

def retrieve_relevant_content(query):
    documents, metadata, scores = fetch_top_relevant_queries(query, collection, top_k=10)
    # Compute cross-encoder scores
    scored_queries = []
    for doc, meta, score in zip(documents, metadata, scores):
        doc_text = doc.replace("\n", " ")
        cross_score = cross_encoder.predict([(query, doc_text)])[0]
        scored_queries.append({"document": doc_text, "metadata": meta, "similarity_score": score, "cross_encoder_score": cross_score})

    # Sort by cross-encoder score and take top 3
    top_docs = sorted(scored_queries, key=lambda x: x['cross_encoder_score'], reverse=True)[:3]

    # Combine top 3 documents into a single paragraph
    combined_text = " ".join([item['document'] for item in top_docs])

    return combined_text


retrieve_relevant_content("What is l and t doing for sustainability for enviornment?")

"view more green business we are not only taking efforts to reduce our own environment footprint but we are also helping our customers create sustainable and green assets. l&tâ\x80\x99s offers a bouquet, termed â\x80\x98green businessâ\x80\x99, centered around clean energy, clean mobility, water and sanitation, green infrastructure, and other areas linked to a greener future. read more 3rd globally in enr's top 200 environmental firms l&t's green commitment has also been acknowledged globally by enr, a globally recognized construction industry publication. l&t has maintained 3rd rank for three consecutive years (2022-2024) in the top 200 environment firms survey by enr. innovation a key lever in our journey to decarbonizing and creating a better world is innovation, driven through r&d efforts, digitalization, sustainable design, and operational improvements. read more technology for sustainable growth view more green business we are not only taking efforts to reduce our own environment

In [21]:
def enhance_prompt(query):
    """Enhances the user query to improve retrieval accuracy."""
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI assistant for L&T group. Your task is to refine user queries for better information retrieval. Make the query clearer, more specific, and structured to help retrieve relevant documents accurately."},
            {"role": "user", "content": f"Enhance this query for better retrieval: {query}"}
        ],
        model="gpt-4o",
        max_tokens=50
    )
    return chat_completion.choices[0].message.content.strip()

def enhanced_llm_call(user_query):
    """Enhances query, retrieves relevant content, and gets final response."""
    
    # Step 1: Enhance the query for better retrieval
    refined_query = enhance_prompt(user_query)

    # Step 2: Retrieve relevant content using the enhanced query
    relevant_content = retrieve_relevant_content(refined_query)

    # Step 3: Send both the retrieved content and the original user question to LLM
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI assistant. Answer the user's question using the given context."},
            {"role": "user", "content": f"Context:\n{relevant_content}\n\nUser Question: {user_query}"}
        ],
        model="gpt-4o",
        max_tokens=512
    )

    return chat_completion.choices[0].message.content

# Example Usage
user_query = "Who is your chairman"
final_output = enhanced_llm_call(user_query)
print(final_output)

The chairman of Larsen & Toubro (L&T) is **Mr. S.N. Subrahmanyan**, who serves as the Chairman and Managing Director. Additionally, **Mr. A.M. Naik** holds the title of Chairman Emeritus.
